In [14]:
# !pip install tzlocal
# pip install python-dateutil
# pip install boto3
import boto3
import base64
import json
import time
import tzlocal

In [20]:
# Initialize a boto3 Kinesis client
kinesis_client = boto3.client('kinesis', region_name='us-east-1')

# The name of your Kinesis stream
stream_name = 'CH_agent_call_streams'

# The sequence number you want to start reading from
sequence_number_for_shard = '49649059939264757358942140911703275325366952989042933794'


In [97]:
# Function to process records and create JSON
def process_records(records):
    for record in records:
        # Kinesis data is UTF-8 encoded so decode here
        payload = record["Data"].decode('utf-8')
        return payload

# Function to get records from Kinesis
def get_kinesis_records(stream_name, sequence_number_for_shard):
    response = kinesis_client.describe_stream(StreamName=stream_name)
    shard_id = response['StreamDescription']['Shards'][2]['ShardId']

    # Get a shard iterator using the sequence number
    shard_iterator = kinesis_client.get_shard_iterator(StreamName=stream_name,
                                                       ShardId=shard_id,
                                                       ShardIteratorType='AT_SEQUENCE_NUMBER',
                                                       StartingSequenceNumber=sequence_number_for_shard)['ShardIterator']

    out = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=100)
    shard_iterator = out['NextShardIterator']
    records = out['Records']

    if records:
        json_data = process_records(records)

    time.sleep(1)  # Sleep to avoid hitting rate limits
    return json_data

In [98]:
response = get_kinesis_records(stream_name, sequence_number_for_shard)

In [99]:
print(response)

[
  {
    Transcript: {
      ParticipantId: 'AGENT',
      ParticipantRole: 'AGENT',
      Content: 'Good morning! Thank you for calling Patient Pathways. My name is Alex, and I'll be assisting you with your enrollment. How may I address you?',
      BeginOffsetMillis: 1257,
      EndOffsetMillis: 9697,
      Sentiment: 'NEUTRAL',
      Id: 'fb29489e-d06b-48b1-8e0b-519f17c4a68e'
    }
  },
  {
    Transcript: {
      ParticipantId: 'CUSTOMER',
      ParticipantRole: 'CUSTOMER',
      Content: 'Hello, Alex. I'm Jordan Smith',
      BeginOffsetMillis: 3047,
      EndOffsetMillis: 8055,
      Sentiment: 'POSITIVE',
      Id: '7f6820ee-0487-4cbc-9b35-33e7c31e1d5d'
    }
  },
  {
    Transcript: {
      ParticipantId: 'AGENT',
      ParticipantRole: 'AGENT',
      Content: ' It's nice to meet you, Jordan. To start with your enrollment, I'll need to ask you a few questions. Is now a good time?',
      BeginOffsetMillis: 10457,
      EndOffsetMillis: 16775,
      Sentiment: 'POSITIVE',
     